In [0]:
from __future__ import division
import heapq, copy

class Graph(object):
    def __init__(self, numVerts, numEdges):
        self.numVerts = numVerts
        self.numEdges = numEdges
        self.edges = []                            # (v1, v2)
        self.edgeCosts = []
        self.verts = [[] for _ in range(numVerts)] # list of adj edges

    def addEdge(self, v1, v2, cost=1):
        edgeIdx = len(self.edges)
        self.edges.append((v1, v2))
        self.edgeCosts.append(cost)
        self.verts[v1].append(edgeIdx)
        self.verts[v2].append(edgeIdx)

    def getEdge(self, idx):
        return self.edges[idx]

    def getEdgeCost(self, idx):
        return self.edgeCosts[idx]

    def setEdgeCost(self, idx, cost):
        self.edgeCosts[idx] = cost
        
    def getVert(self, idx):
        return self.verts[idx]

    def getVertHead(self, idx):
        return [eIdx for eIdx in self.verts[idx]
                if self.edges[eIdx][1] == idx]

    def getVertHeadIter(self, idx):
        for eIdx in self.verts[idx]:
            if self.edges[eIdx][1] == idx:
                yield eIdx
                
    def getVertTail(self, idx):
        return [eIdx for eIdx in self.verts[idx]
                if self.edges[eIdx][0] == idx]

    def getVertTailIter(self, idx):
        for eIdx in self.verts[idx]:
            if self.edges[eIdx][0] == idx:
                yield eIdx
                
    def clone(self):
        cloned = Graph(self.numVerts, self.numEdges)
        cloned.edges = copy.deepcopy(self.edges)
        cloned.edgeCosts = copy.deepcopy(self.edgeCosts)
        cloned.verts = copy.deepcopy(self.verts)
        return cloned
    
    def __repr__(self):
        return "<Graph m=%d, n=%d, edges=%s verts=%s>" % (
            self.numEdges, self.numVerts, zip(self.edges, self.edgeCosts), self.verts)

    @staticmethod
    def loadFromFile(datafile, one_based = False):
        data = []
        f = open(datafile)
        numVerts, numEdges = [int(x) for x in f.readline().strip().split()]
        G = Graph(numVerts, numEdges)
        for line in f.readlines():
            line = line.strip()
            if len(line) == 0:
                continue
            parts = [int(x) for x in line.split()]
            if len(parts) == 3:
                v1, v2, cost = parts
            elif len(parts) == 2:
                v1, v2 = parts
                cost = 1
            else:
                pass
            if one_based:
                G.addEdge(v1-1, v2-1, cost)
            else:
                G.addEdge(v1, v2, cost)
        assert(G.numEdges == len(G.edges))
        f.close()
        return G

class UnionFind(object):
    def __init__(self, numItems):
        self.items = list(range(numItems))
        self.sizes = [[1] for _ in self.items]
        self.numSets = numItems

    def union(self, i, j):
        set_i = self.find(i)
        set_j = self.find(j)
        if set_i != set_j:
            self.numSets -= 1
            if self.sizes[set_i] > self.sizes[set_j]:
                self.items[set_j] = set_i
                self.sizes[set_i] += self.sizes[set_j]
            else:
                self.items[set_i] = set_j
                self.sizes[set_j] += self.sizes[set_i]

    def find(self, i):
        while self.items[i] != i:
            i = self.items[i]
        return self.items[i]

    def asDict(self):
        sets = {}
        for idx, item in enumerate(self.items):
            aSet = sets.setdefault(self.find(item), [])
            aSet.append(idx)
        return sets
        
    def __repr__(self):
        return repr(self.asDict())

In [45]:
from __future__ import division
import heapq

def Prim(G):
    MST = []
    T = set()
    pq = []
    T.add(1)
    for eIdx in G.getVert(1):
        heapq.heappush(pq, (G.getEdgeCost(eIdx), eIdx))
    while pq:
        cost, eIdx = heapq.heappop(pq) 
        v1, v2 = G.getEdge(eIdx)
        if v1 not in T:
            v = v1
        elif v2 not in T:
            v = v2
        else:
            continue
        T.add(v)
        MST.append(eIdx)
        for edge in G.getVert(v):
            heapq.heappush(pq, (G.getEdgeCost(edge), edge))
    return MST
        

def Kruskal(G):
    uf = UnionFind(G.numVerts)
    pq = [(G.getEdgeCost(idx), idx) for idx in range(G.numEdges)]
    heapq.heapify(pq)
    MST = []
    while uf.numSets > 1:
        eCost, eIdx = heapq.heappop(pq)
        s1, s2 = [uf.find(v) for v in G.getEdge(eIdx)]
        if s1 != s2:
            uf.union(s1, s2)
            MST.append(eIdx)
    return MST

# Question 1
g1 = Graph(9,14)
g1.addEdge(0,1,4)
g1.addEdge(0,7,8)
g1.addEdge(1,2,8)
g1.addEdge(1,7,11)
g1.addEdge(2,3,7)
g1.addEdge(2,8,2)
g1.addEdge(2,5,4)
g1.addEdge(3,4,9)
g1.addEdge(3,5,14)
g1.addEdge(4,5,10)
g1.addEdge(5,6,2)
g1.addEdge(6,7,1)
g1.addEdge(6,8,6)
g1.addEdge(7,8,7)
print(g1)
print("Prims")
print(Prim(g1)) 
print("Kruskal")
print(Kruskal(g1))
g2 = Graph(9,14)
g2.addEdge(0,1,10)
g2.addEdge(0,2,12)
g2.addEdge(1,2,9)
g2.addEdge(1,3,8)
g2.addEdge(2,4,3)
g2.addEdge(3,4,7)
g2.addEdge(3,4,3)
g2.addEdge(4,5,1)
g2.addEdge(2,5,1)
g2.addEdge(3,6,8)
g2.addEdge(3,7,5)
g2.addEdge(5,7,6)
g2.addEdge(6,7,9)
g2.addEdge(6,8,2)
g2.addEdge(7,8,11)
print(g2)
print("Prims")
print(Prim(g2)) 
print("Kruskal")
print(Kruskal(g2))

<Graph m=14, n=9, edges=<zip object at 0x7f44616d7488> verts=[[0, 1], [0, 2, 3], [2, 4, 5, 6], [4, 7, 8], [7, 9], [6, 8, 9, 10], [10, 11, 12], [1, 3, 11, 13], [5, 12, 13]]>
Prims
[0, 1, 11, 10, 6, 5, 4, 7]
Kruskal
[11, 5, 10, 0, 6, 4, 1, 7]
<Graph m=14, n=9, edges=<zip object at 0x7f44617121c8> verts=[[0, 1], [0, 2, 3], [1, 2, 4, 8], [3, 5, 6, 9, 10], [4, 5, 6, 7], [7, 8, 11], [9, 12, 13], [10, 11, 12, 14], [13, 14]]>
Prims
[3, 6, 7, 8, 10, 9, 13, 0]
Kruskal
[7, 8, 13, 6, 10, 3, 9, 0]
